A work in progress - beware, dragons!

Feynman-Kleinert-1986-PRA

Workbook to reimplement Feynman and Klenert's 1986 PRA "Effective classical partition function"

https://doi.org/10.1103/PhysRevA.34.5080

Effective classical partition functions. R. P. Feynman and H. Kleinert. Phys. Rev. A 34, 5080 – Published 1 December 1986

Errata

The form for the double well potential (page 34, RHS, third paragraph starting 'Another example is the double-well...'), should read

$V(x)=-\frac{1}{2} x^2 + \frac{1}{4} g x^4 + \frac{1}{4g}$.

Strangely this is correct in the captions of figure 2 and 3!

In [1]:
# Bring out the major leagues... https://www.youtube.com/watch?v=_E6DDktoPhg
using Optim

# Implementing optimisation / minimisation described in (6)
Pkg.status("Optim.jl") # I've probably pinned this to something weird.

 - Optim                         0.7.8              pinned.250b50ab.tmp


In [2]:
# Variables
# Potential energy g; either anharmonic strength or double-well setup
g=0.1976
# Thermodynamic Beta, natch
β=10

10

In [3]:
# Some kind of potential
# Double well potential as given in figure 2 + 3 captions
V(x,g)=-0.5*x^2 + 0.25*g*x^4 + 0.25/g

# Harmonic oscillator
#V(x,g)=0.5*x^2

# Anharmonic potential, following Kleinhert's book (p. 471); Also Figure 1 Feynman-Kleinert
#V(x,g)=x^2/2+x^4*g/4

V (generic function with 1 method)

In [4]:
using QuadGK
# (4) in Feynman and Kleinert
K(xp,x,a2,g)=1/sqrt(2*π*a2)*exp(-(x-xp)^2/(2*a2))*V(xp,g)
Va2(x,a2,g)=quadgk(xp->K(xp,x,a2,g), -Inf, +Inf)

Va2 (generic function with 1 method)

In [5]:
# (5) naively as written, in Feynman and Kleinert
#Wtilde(x0,a2,Ω,g,β)=(1/β) * log( sinh(β*Ω/2)/(β*Ω/2) ) - (Ω^2/2) * a2 + Va2(x0,a2,g)[1]

# Large β limit; by taking sinh(x)=e^x-e^-x , and then dropping e^-x
#Wtilde(x0,a2,Ω,g,β)=(1/β) * (β*Ω/2 - log(β*Ω/2)) - (Ω^2/2) * a2 + Va2(x0,a2,g)[1]

# Branch depending on β; for accuracy + lack of infinities
function Wtilde(x0,a2,Ω,g,β)
    D=β*Ω/2
    if D>7.5
        F=(1/β) * (D - log(2*D))
    else
        F=(1/β)*( log(sinh(D)/D) )
    end
    V=Va2(x0,a2,g)[1]
    mid=(Ω^2/2) * a2
    return F-mid+V
end

# Expanded version of the above, reporting the decomposition
function verboseWtilde(x0,a2,Ω,g,β)
    D=β*Ω/2
    println("D=β*Ω/2 = $D")
    F= (1/β)*( log(sinh(D)) - log(D) )
    #F= (1/β)*( (log(2D)+D^2/6+D^4/180) - log(D) ) # generalised Puiseux series
    # Large D limit, through sinh(D) --> e^D [through away e^-D part]
    F = (1/β) * (D - log(2*D))
    V=Va2(x0,a2,g)[1]
    mid=(Ω^2/2) * a2
    println("Wtilde(x0=$x0,a2=$a2,Ω=$Ω,g=$g,β=$β) \n\t= F - mid + V")
    println("\t= $F - $mid + $V = ",F-mid+V)
    return F-mid+V
end

verboseWtilde(1.943,0.397,1.255,0.1976,β) # Kleinert's book, Table 5.1 p. 467, for g/4=0.5

verboseWtilde(0.0,0.1306,3.829,40.0,4000000)

D=β*Ω/2 = 6.2749999999999995
Wtilde(x0=1.943,a2=0.397,Ω=1.255,g=0.1976,β=10) 
	= F - mid + V
	= 0.3745279334422207 - 0.31264246249999994 + 0.3507256576575897 = 0.4126111285998104
D=β*Ω/2 = 7.658e6
Wtilde(x0=0.0,a2=0.1306,Ω=3.829,g=40.0,β=4000000) 
	= F - mid + V
	= 1.914495863897852 - 0.9573790373000001 + 0.45264080000000184 = 1.4097576265978538


1.4097576265978538

In [6]:
# Test the approximations used above... graphically like a physicist

using Plots

xrange=0.1:1.0:10

Fnaive(x)=log( sinh(x)/(x) )
Flarge(x)=x-log(2x)

plot(Fnaive,xrange,label="Fnaive")
plot!(Flarge,xrange,label="Flarge")



<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 
 
 
 
 
 Fnaive 
 
 
 
 Flarge

In [7]:
xrange=0.1:1.0:10

plot(x->Flarge(x)-Fnaive(x),xrange,label="Flarge-Fnaive")
#plot!(ylim=(0.1,10))
yaxis!("Error",:log10)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10 
 
 
 - 
 
 
 6 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 0 
 
 
 Error 
 
 
 
 
 
 
 Flarge-Fnaive

In [8]:
println("V(0.0) = ",V(0.0,g))
println("Va2(0.0,5.0) = ",Va2(0.0,1.0,g))

V(0.0) = 1.2651821862348178
Va2(0.0,5.0) = (0.9133821862348174, 1.1458092767843961e-8)


In [9]:
using Plots

In [10]:
# OK, let's have a look our actors:
# V, the bare potential
# Va2, the Gaussian-smoothed potential
# Wtilde, the Auxillary potential, including partition function magic / failure

xrange=-3:0.1:3

plot(x->V(x,g),xrange,label="V") # bare [potential]

for a2 in 0.5 #:0.1:0.5 # Gaussian smearing widths to apply
   plot!(x->Va2(x,a2,g)[1],xrange,label="Va2, a2=$a2")
   plot!(x->Wtilde(x,a2,5,g,β),xrange,label="Wtilde, a2=$a2") # bare [potential]
end

#plot!(ylim=(0,1)) # force display
plot!()

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,81.5963 48.5827,95.5687 57.7953,106.919 67.0079,115.87 76.2205,122.635 85.4331,127.422 94.6457,130.428 103.858,131.846 113.071,131.859 122.283,130.641 
 131.496,128.361 140.709,125.179 149.921,121.246 159.134,116.707 168.346,111.698 177.559,106.348 186.772,100.778 195.984,95.1002 205.197,89.421 214.409,83.8374 
 223.622,78.4392 232.835,73.3081 242.047,68.5183 251.26,64.136 260.472,60.2198 269.685,56.8202 278.898,53.9803 288.11,51.7351 297.323,50.1119 306.535,49.1302 
 315.748,48.8016 324.961,49.1302 334.173,50.1119 343.386,51.7351 352.598,53.9803 361.811,56.8202 371.024,60.2198 380.236,64.136 389.449,68.5183 398.661,73.3081 
 407.874,78.4392 417.087,83.8374 426.299,89.421 435.512,95.1002 444.724,100.778 453.937,106.348 463.15,111.698 472.362,116.707 481.575,121.246 490.787,125.179 
 500,128.361 509.213,130.641 518.425,131.859 527.638,131.846 536.85,130.428 546.063,127.422 555.276,122.635 564.488,115.87 573.701,106.919 582.913,95.5687 
 592.126,81.5963 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,7.87402 48.5827,27.5975 57.7953,44.5041 67.0079,58.8159 76.2205,70.7474 85.4331,80.5053 94.6457,88.2884 103.858,94.2878 113.071,98.6867 122.283,101.661 
 131.496,103.377 140.709,103.996 149.921,103.67 159.134,102.542 168.346,100.75 177.559,98.422 186.772,95.6786 195.984,92.6332 205.197,89.3909 214.409,86.0493 
 223.622,82.698 232.835,79.419 242.047,76.2863 251.26,73.3661 260.472,70.7171 269.685,68.3898 278.898,66.4272 288.11,64.8643 297.323,63.7285 306.535,63.0392 
 315.748,62.8081 324.961,63.0392 334.173,63.7285 343.386,64.8643 352.598,66.4272 361.811,68.3898 371.024,70.7171 380.236,73.3661 389.449,76.2863 398.661,79.419 
 407.874,82.698 417.087,86.0493 426.299,89.3909 435.512,92.6332 444.724,95.6786 453.937,98.422 463.15,100.75 472.362,102.542 481.575,103.67 490.787,103.996 
 500,103.377 509.213,101.661 518.425,98.6867 527.638,94.2878 536.85,88.2884 546.063,80.5053 555.276,70.7474 564.488,58.8159 573.701,44.5041 582.913,27.5975 
 592.126,7.87402 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#3da44d; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,280.256 48.5827,299.979 57.7953,316.886 67.0079,331.198 76.2205,343.129 85.4331,352.887 94.6457,360.67 103.858,366.669 113.071,371.068 122.283,374.042 
 131.496,375.759 140.709,376.378 149.921,376.052 159.134,374.924 168.346,373.132 177.559,370.804 186.772,368.06 195.984,365.015 205.197,361.773 214.409,358.431 
 223.622,355.08 232.835,351.801 242.047,348.668 251.26,345.748 260.472,343.099 269.685,340.771 278.898,338.809 288.11,337.246 297.323,336.11 306.535,335.421 
 315.748,335.19 324.961,335.421 334.173,336.11 343.386,337.246 352.598,338.809 361.811,340.771 371.024,343.099 380.236,345.748 389.449,348.668 398.661,351.801 
 407.874,355.08 417.087,358.431 426.299,361.773 435.512,365.015 444.724,368.06 453.937,370.804 463.15,373.132 472.362,374.924 481.575,376.052 490.787,376.378 
 500,375.759 509.213,374.042 518.425,371.068 527.638,366.669 536.85,360.67 546.063,352.887 555.276,343.129 564.488,331.198 573.701,316.886 582.913,299.979 
 592.126,280.256 
 "/>
 
 
 
 
 V 
 
 
 
 Va2, a2=0.5 
 
 
 
 Wtilde, a2=0.5

In [11]:
@printf("Wtilde(x0,a2,Ω,g=%f,β=%f)\n",g,β)

@printf("a2\\Ω")
for Ω in 0.1:0.2:2.0
    @printf(" %.3f",Ω)
end

for a2 in 0.1:0.1:2.0
    @printf("\n a2=%.3f",a2)
    for Ω in 0.1:0.2:2.0
        @printf(" %.3f",Wtilde(0.0,a2,Ω,g,β))
    end
end

Wtilde(x0,a2,Ω,g=0.197600,β=10.000000)
a2\Ω 0.100 0.300 0.500 0.700 0.900 1.100 1.300 1.500 1.700 1.900
 a2=0.100 1.220 1.247 1.293 1.347 1.406 1.466 1.526 1.583 1.639 1.692
 a2=0.200 1.174 1.197 1.234 1.277 1.320 1.360 1.396 1.425 1.449 1.466
 a2=0.300 1.131 1.150 1.179 1.210 1.237 1.257 1.269 1.270 1.262 1.243
 a2=0.400 1.091 1.106 1.127 1.146 1.157 1.157 1.144 1.118 1.078 1.022
 a2=0.500 1.054 1.065 1.078 1.085 1.080 1.060 1.023 0.969 0.896 0.805
 a2=0.600 1.020 1.027 1.032 1.027 1.006 0.966 0.905 0.823 0.718 0.591
 a2=0.700 0.988 0.991 0.989 0.972 0.935 0.875 0.790 0.679 0.543 0.380
 a2=0.800 0.960 0.959 0.948 0.919 0.866 0.786 0.678 0.539 0.371 0.172
 a2=0.900 0.935 0.930 0.911 0.870 0.801 0.701 0.568 0.402 0.201 -0.034
 a2=1.000 0.913 0.903 0.877 0.824 0.739 0.619 0.462 0.268 0.035 -0.236
 a2=1.100 0.893 0.880 0.845 0.780 0.679 0.539 0.359 0.136 -0.128 -0.435
 a2=1.200 0.877 0.860 0.817 0.740 0.623 0.463 0.258 0.008 -0.289 -0.632
 a2=1.300 0.863 0.842 0.792 0.702 0.569 0.389 0.16

In [19]:
# Replot fits from Feynman-Kleinert Table II
# - basically checking whether the Wtilde equation is reporting correctly, 
# and whether (as described in the caption) only Wtilde for g=0.1976 has double well structure 

β=3000
# Much higher than this, and results collapse to Inf. But essentially agrees with Table II now!

# Wtilde(x0,a2,Ω)
xrange=-4:0.1:4
# These values, from Feynman and Kleinhert, table II
g=0.1976
@printf("\n%f %f",g,Wtilde(1.943,0.397,1.255,g,β))
plot(x->Wtilde(x,0.397,1.255,g,β),xrange,label="g=0.1976") 

g=0.4
@printf("\n%f %f",g,Wtilde(0.0,1.030,0.486,g,β))
plot!(x->Wtilde(x,1.030,0.486,g,β),xrange,label="g=0.4") 

g=4.0
@printf("\n%f %f",g,Wtilde(0.0,0.3059,1.634,g,β))
plot!(x->Wtilde(x,0.3059,1.634,g,β),xrange,label="g=4.0")

g=40.0
@printf("\n%f %f",g,Wtilde(0.0,0.1306,3.829,g,β))
plot!(x->Wtilde(x,0.1306,3.829,g,β),xrange,label="g=40.0")
plot!(ylim=(-2,10))

@printf("\nFeynman-Kleinert Table II reads:\n 0.1976 0.650 \n 0.4  0.549 \n 4.0 0.598 \n 40.0 1.409")


0.197600 0.662839
0.400000 0.547201
4.000000 0.596072
40.000000 1.406645
Feynman-Kleinert Table II reads:
 0.1976 0.650 
 0.4  0.549 
 4.0 0.598 
 40.0 1.409

In [23]:
# Wtilde(x0,a2,Ω)
g=0.1976
β=300
myf(x) = Wtilde(0.0,x[1],x[2],g,β)

lower=[0.0,0.0]
upper=[10.0,10.0]
initial=[1.0,1.0]

res=optimize(DifferentiableFunction(myf), 
    initial, lower, upper, Fminbox(); 
    optimizer=GradientDescent, 
    optimizer_o=(Optim.Options(autodiff=true)) )

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] DifferentiableFunction(::Function, ::Vararg{Function,N} where N) at ./deprecated.jl:57
 [3] include_string(::String, ::String) at /Applications/Julia-0.6.app/Contents/Resources/julia/lib/julia/sys.dylib:?
 [4] include_string(::Module, ::String, ::String) at /Users/jarvist/.julia/v0.6/Compat/src/Compat.jl:407
 [5] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/jarvist/.julia/v0.6/IJulia/src/execute_request.jl:154
 [6] eventloop(::ZMQ.Socket) at /Users/jarvist/.julia/v0.6/IJulia/src/eventloop.jl:8
 [7] (::IJulia.##14#17)() at ./task.jl:335
while loading In[23], in expression starting on line 10


Results of Optimization Algorithm
 * Algorithm: Fminbox with Gradient Descent
 * Starting Point: [1.0,1.0]
 * Minimizer: [9.999999999988574,9.999999999994541]
 * Minimum: -4.839415e+02
 * Iterations: 3
 * Convergence: true
   * |x - x'| < 1.0e-32: false
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: true
   * |g(x)| < 1.0e-08: false
   * f(x) > f(x'): false
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 212
 * Gradient Calls: 212

In [14]:
myf((0.2,0.3))

0.22123253507889207

In [15]:
function W(x0,g,β)
    myf(x)=Wtilde(x0,x[1],x[2],g,β)
    
    lower=[0.0,0.0]
    upper=[100.0,100.0]
    initial=[1.0,1.0]

    res=optimize(DifferentiableFunction(myf), 
        initial, lower, upper, Fminbox(); 
        optimizer=GradientDescent, 
        optimizer_o=(Optim.Options(autodiff=true,allow_f_increases=true)) )
    minimum=Optim.minimum(res)
    show(res)
    return minimum
end

# Vaguely trying to produce plateau values on Fig 2; and figures in Tab II
β=10
for g in [0.1976, 0.4,0.4,40]
    @printf("\ng=%f β=%f, W(0.0,g,β)= %f",g,β,W(0.0,g,β))
end

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] DifferentiableFunction(::Function, ::Vararg{Function,N} where N) at ./deprecated.jl:57
 [3] W(::Float64, ::Float64, ::Int64) at ./In[15]:8
 [4] macro expansion at ./In[15]:20 [inlined]
 [5] anonymous at ./<missing>:?
 [6] include_string(::String, ::String) at /Applications/Julia-0.6.app/Contents/Resources/julia/lib/julia/sys.dylib:?
 [7] include_string(::Module, ::String, ::String) at /Users/jarvist/.julia/v0.6/Compat/src/Compat.jl:407
 [8] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/jarvist/.julia/v0.6/IJulia/src/execute_request.jl:154
 [9] eventloop(::ZMQ.Socket) at /Users/jarvist/.julia/v0.6/IJulia/src/eventloop.jl:8
 [10] (::IJulia.##14#17)() at ./task.jl:335
while loading In[15], in expression starting on line 19


Results of Optimization Algorithm
 * Algorithm: Fminbox with Gradient Descent
 * Starting Point: [1.0,1.0]
 * Minimizer: [99.9999999998702,99.99999999993547]
 * Minimum: -4.985174e+05
 * Iterations: 2
 * Convergence: true
   * |x - x'| < 1.0e-32: false
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: true
   * |g(x)| < 1.0e-08: false
   * f(x) > f(x'): false
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 128
 * Gradient Calls: 128
g=0.197600 β=10.000000, W(0.0,g,β)= -498517.425592Results of Optimization Algorithm
 * Algorithm: Fminbox with Gradient Descent
 * Starting Point: [1.0,1.0]
 * Minimizer: [99.99999999989981,99.9999999999505]
 * Minimum: -4.970001e+05
 * Iterations: 2
 * Convergence: true
   * |x - x'| < 1.0e-32: false
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: true
   * |g(x)| < 1.0e-08: false
   * f(x) > f(x'): false
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 126
 * Gradient Calls: 126
g=0.400000 β=10.000000, W(0.0,g,β)= 

In [16]:
xrange=-0.4:0.05:0.5
#Ws=[ W(x) for x in xrange ]
#println(Ws)
#plot(Ws,xrange,label="W(x)")
plot(x->W(x,g,β), xrange, label="W(x)")

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] DifferentiableFunction(::Function, ::Vararg{Function,N} where N) at ./deprecated.jl:57
 [3] W(::Float64, ::Float64, ::Int64) at ./In[15]:8
 [4] _collect(::StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}}, ::Base.Generator{StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}},##20#21}, ::Base.EltypeUnknown, ::Base.HasShape) at ./array.jl:454
 [5] map(::Function, ::StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}}) at ./abstractarray.jl:1865
 [6] compute_xyz(::StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}}, ::Function, ::Void) at /Users/jarvist/.julia/v0.6/Plots/src/series.jl:93
 [7] macro expansion at /Users/jarvist/.julia/v0.6/Plots/src/series.jl:150 [inlined]
 [8] apply_recipe(::Dict{Symbol,Any}, ::Type{Plots.SliceIt}, ::StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float6

LoadError: DomainError:

In [17]:
W(1.0,g,β)

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] DifferentiableFunction(::Function, ::Vararg{Function,N} where N) at ./deprecated.jl:57
 [3] W(::Float64, ::Float64, ::Int64) at ./In[15]:8
 [4] include_string(::String, ::String) at /Applications/Julia-0.6.app/Contents/Resources/julia/lib/julia/sys.dylib:?
 [5] include_string(::Module, ::String, ::String) at /Users/jarvist/.julia/v0.6/Compat/src/Compat.jl:407
 [6] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/jarvist/.julia/v0.6/IJulia/src/execute_request.jl:154
 [7] eventloop(::ZMQ.Socket) at /Users/jarvist/.julia/v0.6/IJulia/src/eventloop.jl:8
 [8] (::IJulia.##14#17)() at ./task.jl:335
while loading In[17], in expression starting on line 1


LoadError: DomainError:

In [18]:
sinh(700)

5.0711602736750225e303